In [1]:
from google.colab import drive
import os

drive.mount("/content/drive/")
os.chdir("/content/drive/MyDrive/Colab Notebooks/GeoDiv")

Mounted at /content/drive/


In [2]:
!ls -al

total 217
-rw------- 1 root root   5777 Nov 17 13:18 data_utils.ipynb
-rw------- 1 root root   7619 Dec  7 13:04 evaluate.ipynb
-rw------- 1 root root  23355 Dec 12 10:46 GeoDiv.ipynb
drwx------ 2 root root   4096 Nov 18 06:21 newlossModel
drwx------ 2 root root   4096 Oct 31 06:25 Philadelphia
-rw------- 1 root root  10820 Dec 12 11:12 Preprocess.ipynb
-rw------- 1 root root  24533 Dec 12 09:00 test_Yelp.ipynb
-rw------- 1 root root 131906 Dec 12 11:11 train_Yelp.ipynb
drwx------ 2 root root   4096 Oct 31 07:00 Tucson
drwx------ 2 root root   4096 Nov 17 13:20 Yelp


In [3]:
import collections
import json

In [ ]:
user_dict = collections.defaultdict(set)
poi_dict = collections.defaultdict(set)

with open('./Tucson/Tucson.json', mode="r") as f:

  line = f.readline()

  while line:

    line = json.loads(line)

    user_dict[line['user_id']].add(line['business_id'])
    poi_dict[line['business_id']].add(line['user_id'])

    line = f.readline()

f.close()

In [ ]:
print(len(user_dict))
print(len(poi_dict))

103342
8809


In [ ]:
delete_user = []
delete_poi = []

for k,v in user_dict.items():
  if len(v) < 15:
    delete_user.append(k)

for k,v in poi_dict.items():
  if len(v) < 10:
    delete_poi.append(k)

In [ ]:
print(len(user_dict) - len(delete_user))
print(len(poi_dict) - len(delete_poi))

2419
5147


In [ ]:
user_list = list(user_dict.keys())
user_map = list(set(user_list).difference(set(delete_user)))
print(len(user_map))

poi_list = list(poi_dict.keys())
poi_map = list(set(poi_list).difference(set(delete_poi)))
print(len(poi_map))

4716
8075


In [ ]:
# user_map = dict()
# poi_map = dict()

# with open('./Philadelphia/Philadelphia_check.json', mode="r") as f:

#   line = f.readline()
  
#   while line:

#     line = json.loads(line)

#     user_map[line['user_name']] = line['user_id']
#     poi_map[line['poi_name']] = line['poi_id']

#     line = f.readline()
  
# f.close()


In [ ]:
# with open('./yelp_academic_dataset_business.json', mode="r") as f:
#   with open("./Philadelphia/Philadelphia_location.json", mode="w") as ff:

#     line = f.readline()
    
#     while line:

#       sample = json.loads(line)

#       if sample['business_id'] in poi_map:

#         temp = dict()
#         temp['poi_id'] = poi_map[sample['business_id']]
#         temp["latitude"] = float(sample["latitude"])
#         temp["longitude"] = float(sample["longitude"])

#         temp = json.dumps(temp)
#         ff.write(temp + "\n")
      
#       line = f.readline()


#   ff.close()
# f.close()

In [ ]:
# with open('./Philadelphia_location.txt', mode='w') as f:

#   for sample in POI:
    
#     temp = str(sample['business_id'])
#     temp += " "
#     temp += str(sample["latitude"])
#     temp += " "
#     temp += str(sample["longitude"])
#     temp += "\n"

#     f.write(temp)

# f.close()

In [ ]:
# All = []

# xxx = set()
# yyy = set()

# with open('./Philadelphia/Philadelphia.json', mode="r") as f:

#   line = f.readline()

#   while line:

#     sample = json.loads(line)

#     if sample['user_id'] in user_map and sample['business_id'] in poi_map:
      
#       xxx.add(sample['user_id'])
#       yyy.add(sample['business_id'])

#       All.append(sample)

#       # sample = json.dumps(sample)
#       # f.write(sample + "\n")

#     line = f.readline()

# f.close()

# All = sorted(All, key=lambda x:x['date'])

In [ ]:
# print(len(xxx))
# print(len(yyy))
# print(len(All))

4716
7727
173644


In [ ]:
# xxx = list(xxx)
# yyy = list(yyy)

# with open('./Philadelphia/Philadelphia_check.json', mode="w") as f:

#   for sample in All:

#     temp = dict()

#     temp['user_name'] = sample['user_id']
#     temp['user_id'] = xxx.index(sample['user_id'])
#     temp['poi_name'] = sample['business_id']
#     temp['poi_id'] = yyy.index(sample['business_id'])
#     temp['date'] = sample['date']

#     temp = json.dumps(temp)
#     f.write(temp + "\n")

# f.close()

In [6]:
DataSet = collections.defaultdict(list)

with open('./Philadelphia/Philadelphia_check.json', mode='r') as f:

  line = f.readline()

  while line:

    sample = json.loads(line)
    
    DataSet[sample['user_id']].append(sample['poi_id'])

    line = f.readline()

f.close()

In [10]:
import numpy as np

for k,v in DataSet.items():
  DataSet[k] = set(v)
  

np.save("./Philadelphia/user_rating_set_all.npy", DataSet, allow_pickle=True)

In [ ]:
trainSet = collections.defaultdict(list)
testSet = collections.defaultdict(list)
validSet = collections.defaultdict(list)


for k,v in DataSet.items():
  
  l = len(v)

  train = int(round(l*0.7))
  test = int(round(l*0.1))
  valid = int(round(l*0.2))

  l_ = l - (train + test + valid)

  if l_ == -1:
    train = train - 1
  if l_ == 1:
    test = test + 1
  
  trainSet[k] = v[0:train]
  testSet[k] = v[train:train+test]
  validSet[k] = v[train+test:]

for k,v in DataSet.items():

  x = list(set(trainSet[k]) & set(testSet[k]))
  y = list(set(trainSet[k]) & set(validSet[k]))

  trainSet[k] = list(set(trainSet[k]).difference(set(x)).difference(set(y)))
  testSet[k] = list(set(testSet[k]).difference(set(x)))
  validSet[k] = list(set(validSet[k]).difference(set(y)))

In [ ]:
print(trainSet[99])
print(testSet[99])
print(validSet[99])

[258, 2436, 2956, 4044, 7567, 3030, 4823, 5528, 30, 4840, 1577, 6890, 6324, 1653, 311, 6200]
[4745]
[5185, 649, 814, 398, 6140]


In [ ]:
import numpy as np

np.save("./Philadelphia/testList.npy", testSet, allow_pickle=True)

XXX = np.load("./Philadelphia/trainList.npy", allow_pickle=True).item()

print(type(XXX))

<class 'collections.defaultdict'>
